# EJERCICIO 5 - Arredondo Vazquez Guillermo
Inicializa una sesión de spark en consola y coloca en una variable llamada playersPersInfo la
información del archivo t_players_info.csv, t_players_salary.csv, t_players_pers_info.csv
considerando la realización de un left join; usa como llave primaria el campo Rank y muestra el
resultado en pantalla.
- Muestra el tipo de dato de cada columna.
- Filtra solo por los jugadores que no viven en los Estados Unidos
- Ordena los jugadores del más joven al más veterano
- Escribe el resultado en un nuevo CSV delimitado por pipes “|” y nómbralo
csv_result_players_info.csv

In [1]:
#Para utilizar pyspark en notebook es necesario utilizar findspark
#con la funcion .init() lo inicializamos
import findspark
findspark.init()
import pandas as pd
import pyspark

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date
#Libreria para utilizar el count
from pyspark.sql.functions import *

In [2]:
#El sparkSession crea internamente sparkconfig y el sparkContext
spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_Df')\
        .getOrCreate()

In [62]:
#Muestra el tipo de dato de cada columna
dfInfo =spark.read.csv("data/t_players_info.csv", sep=",", header=True, inferSchema=True)
dfSal = spark.read.csv("data/t_players_salary.csv", sep="|", header=True, inferSchema=True)
dfPer = spark.read.csv("data/t_players_pers_info.csv", sep=",", header=False, inferSchema=True)

dfPer2 = dfPer.withColumnRenamed("_c0", "Rank")\
.withColumnRenamed("_c1", "Country").withColumnRenamed("_c2", "Year")\
.withColumnRenamed("_c3", "Date").withColumnRenamed("_c4", "Home")\
.withColumnRenamed("_c5", "Winnings")

playersPersInfo = dfInfo.join(dfSal, ["Rank"], "left").join(dfPer2,["Rank"],"left")
playersPersInfo.show(10)


+----+---------------+--------+--------------------+------+-----------+---------------+------------+------------------+----+----------+--------------------+--------+
|Rank|           Name|   Sport|      Wikipedia Page|Gender|  Total Pay|Salary/Winnings|Endorsements|           Country|Year|      Date|                Home|Winnings|
+----+---------------+--------+--------------------+------+-----------+---------------+------------+------------------+----+----------+--------------------+--------+
|  55|  Aaron Rodgers|Football|http://en.wikiped...|  Male|$22,000,000|    $11,000,000| $11,000,000|     United States|1981|26/09/1981|Saginaw, Michigan...|     175|
|  55|  Aaron Rodgers|Football|http://en.wikiped...|  Male|$22,000,000|    $11,000,000| $11,000,000|     United States|1983| 2/12/1983|Chico, California...|     188|
|  55|  Aaron Rodgers|Football|http://en.wikiped...|  Male|$22,000,000|    $14,500,000|  $7,500,000|     United States|1981|26/09/1981|Saginaw, Michigan...|     175|
|  5

In [54]:
#Muestra el tipo de dato de cada columna.
playersPersInfo.printSchema()


root
 |-- Rank: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sport: string (nullable = true)
 |-- Wikipedia Page: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Total Pay: string (nullable = true)
 |-- Salary/Winnings: string (nullable = true)
 |-- Endorsements: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Date: string (nullable = true)
 |-- Home: string (nullable = true)
 |-- Winnings: integer (nullable = true)



In [80]:
#Filtra solo por los jugadores que no viven en los Estados Unidos
noUSA = playersPersInfo.filter(playersPersInfo["Country"] != 'United States')
noUSA.show(10)


+----+---------------+----------+--------------------+------+-----------+---------------+------------+------------------+----+----------+--------------------+--------+
|Rank|           Name|     Sport|      Wikipedia Page|Gender|  Total Pay|Salary/Winnings|Endorsements|           Country|Year|      Date|                Home|Winnings|
+----+---------------+----------+--------------------+------+-----------+---------------+------------+------------------+----+----------+--------------------+--------+
|  95|     Adam Scott|      Golf|https://en.wikipe...|  Male|$17,700,000|     $8,700,000|  $9,000,000|         Australia|1980|16/07/1980| Adelaide, Australia|     183|
|  60|Adrian Gonzalez|  Baseball|http://en.wikiped...|  Male|$21,500,000|    $18,000,000|  $3,500,000|Dominican Republic|1982|22/11/1982|San Pedro de Maco...|     183|
|  60|Adrian Gonzalez|  Baseball|http://en.wikiped...|  Male|$21,500,000|    $21,100,000|    $400,000|Dominican Republic|1982|22/11/1982|San Pedro de Maco...|  

In [81]:
#Ordena los jugadores del más joven al más veterano
playersPersInfo.sort(desc("Year")).show(10)
# Mostrar el DataFrame ordenado


+----+--------------+--------+--------------------+------+-----------+---------------+------------+-------+----+----------+--------------------+--------+
|Rank|          Name|   Sport|      Wikipedia Page|Gender|  Total Pay|Salary/Winnings|Endorsements|Country|Year|      Date|                Home|Winnings|
+----+--------------+--------+--------------------+------+-----------+---------------+------------+-------+----+----------+--------------------+--------+
|  16|        Neymar|  Soccer|http://en.wikiped...|  Male|$33,600,000|    $17,600,000| $16,000,000| Brazil|1992|05/02/1992|Mogi das Cruzes, ...|     175|
|  66|   Eli Manning|Football|http://en.wikiped...|  Male|$21,000,000|    $13,500,000|  $7,500,000| Mexico|1990|18/07/1990|Guadalajara, Jali...|     180|
|  66|   Eli Manning|Football|http://en.wikiped...|  Male|$21,000,000|    $19,500,000|  $1,500,000| Mexico|1990|18/07/1990|Guadalajara, Jali...|     180|
|  66|Canelo Alvarez|  Boxing|http://en.wikiped...|  Male|$21,000,000|    $1

In [79]:
#Escribe el resultado en un nuevo CSV delimitado por pipes “|” y nómbralo csv_result_players_info.csv
output_path = 'C:/Users/SoporteTI/Entrega/Ejercicios_para_Entregar/csv_result_players_info.csv'
playersPersInfo.write.csv(output_path, sep="|", header=True, mode="overwrite")